In [1]:
import pandas as pd
import glob
import urllib.parse
import whois
from datetime import datetime, timedelta
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor, as_completed
import socket
import re

In [6]:
import re

#1. 정규 표현식으로 IP 주소 형식 패턴 정의(범주X)
ip_pattern = r"(?:\d{1,3}\.){3}\d{1,3}"

# 'IP_LIKE' 열 추가
df['IP_LIKE'] = pd.Series(dtype=int)

# URL 문자열 내 IP 주소 형식 존재 여부 확인 및 'IP_LIKE' 열 업데이트
for idx, url in df['url'].items():
    if re.search(ip_pattern, url):
        df.loc[idx, 'IP_LIKE'] = 1
    else:
        df.loc[idx, 'IP_LIKE'] = 0

In [7]:
#2.Checks the presence of @ in URL (HaveAt)(범주X)
df.loc[:, 'AT'] = pd.Series(dtype=int)

def HaveAt(url):
    if "@" in url:
        at = 1
    else:
        at = 0
    return at

# Apply the function to each URL and update 'AT' column
df.loc[:, 'AT'] = df['url'].apply(HaveAt)

In [8]:
#3. Gives number of '/' in URL (URL_Depth)(범주X
from urllib.parse import urlparse
import pandas as pd

# DataFrame에서 'URL_Depth' 컬럼 생성
df.loc[:, 'URL_Depth'] = pd.Series(dtype=int)

def getDepth(url):
    path = urlparse(url).path
    # 슬래시로 분할하여 유효한 부분의 깊이를 계산
    segments = [segment for segment in path.split('/') if segment]
    depth = len(segments)

    # 깊이를 -1, 0, 1로 제한
    if depth == 0:
        return -1
    elif depth == 1:
        return 0
    else:
        return 1

# Apply the function to each URL and update 'URL_Depth' column
df.loc[:, 'URL_Depth'] = df['url'].apply(getDepth)

In [9]:
#4.Checking for redirection '//' in the url (Redirection)

# 새로운 'Redirection' 열을 추가하고 int 형으로 초기화(범주 0)
df.loc[:, 'Redirection'] = pd.Series(dtype=int)

def redirection(url):
    pos = url.rfind('//')  # URL 내에서 마지막으로 나타나는 '//'의 위치를 찾기
    if pos > 6:  # 위치가 6보다 크다면
        if pos > 7:  # 위치가 7보다 크다면 (즉, 프로토콜 부분을 넘어서 존재하는 경우)
            return 1  # 리디렉션이 의심되는 URL로 간주하여 1 반환
        else:
            return 0  # 그렇지 않다면 0 반환(의심)
    else:
        return 0  # '//'가 프로토콜 부분에만 있는 경우 0 반환(의심)

# 각 URL에 대해 redirection 함수를 적용하고 'Redirection' 열을 업데이트
df.loc[:, 'Redirection'] = df['url'].apply(redirection)

In [10]:
#5.URL의 스킴이 'https'인지(범주X)
import urllib.parse

def is_https(url):
    return 1 if urllib.parse.urlsplit(url).scheme == 'https' else 0

df.loc[:, 'Is_Https'] = df['url'].apply(is_https)

In [11]:
import re

#6.1 20가지의 주요한 url 단축 서비스 패턴
shorteningServices = r"bit\.ly|goo\.gl|shorte\.st|go2l\.ink|x\.co|ow\.ly|t\.co|tinyurl|tr\.im|is\.gd|cli\.gs|" \
                      r"yfrog\.com|migre\.me|ff\.im|tiny\.cc|url4\.eu|twit\.ac|su\.pr|twurl\.nl|snipurl\.com|" \
                      r"short\.to|BudURL\.com|ping\.fm|post\.ly|Just\.as|bkite\.com|snipr\.com|fic\.kr|loopt\.us|" \
                      r"doiop\.com|short\.ie|kl\.am|wp\.me|rubyurl\.com|om\.ly|to\.ly|bit\.do|t\.co|lnkd\.in|db\.tt|" \
                      r"qr\.ae|adf\.ly|goo\.gl|bitly\.com|cur\.lv|tinyurl\.com|ow\.ly|bit\.ly|ity\.im|q\.gs|is\.gd|" \
                      r"po\.st|bc\.vc|twitthis\.com|u\.to|j\.mp|buzurl\.com|cutt\.us|u\.bb|yourls\.org|x\.co|" \
                      r"prettylinkpro\.com|scrnch\.me|filoops\.info|vzturl\.com|qr\.net|1url\.com|tweez\.me|v\.gd|" \
                      r"tr\.im|link\.zip\.net|buff\.ly|rb\.gy|rebrand\.ly|short\.cm|clk\.im|cutt\.ly|t2m\.io|bl\.ink|" \
                      r"tiny\.cc"    #단축서비스 패턴 10개 추가함

df.loc[:, 'TINY_URL'] = pd.Series(dtype=int)

#6.2 url에 단축서비스가 포함되어있는 지 확인(범주X)
def tinyURL(url):
    match=re.search(shorteningServices,url) #정규 표현식을 사용하여 입력된 url에서 단축서비스 패턴을 찾음
    if match: #입력된 url에 단축서비스 패턴이 있으면 1을 리턴, 아니면 0을 리턴
        return 1
    else:
        return 0

# Apply the function to each URL and update 'TINY_URL' column
df.loc[:, 'TINY_URL'] = df['url'].apply(tinyURL)

In [12]:
#7. -이 있는지 (범주X)
def check_Hyphen(domain):
    if '-' in domain:
        return 1 
    else:
        return 0 

df['Check_Hyphen'] = df['url'].apply(check_Hyphen)

In [13]:
#8.쿼리문자열의 개수 (범주X)
def parse_query_string(url):
    # URL에서 '?' 문자열이 있는지 확인하여 쿼리 문자열이 있는지 판별
    if '?' not in url:
        return 0

    # URL에서 쿼리 문자열 추출
    query_string = url.split('?')[-1]

    # 쿼리 문자열을 '&'로 분할하여 각 쌍을 추출
    query_pairs = query_string.split('&')

    # 각 쌍을 이름과 값으로 분할하여 딕셔너리에 저장
    params = {}
    for pair in query_pairs:
        if '=' in pair:
            key, value = pair.split('=', 1)
            params[key] = value
        else:
            params[pair] = None

    return len(params)

def categorize_query_count(count):
    if count == 0:
        return -1
    elif count == 1:
        return 0
    else:
        return 1

def Query(url):
    query_count = parse_query_string(url)
    return categorize_query_count(query_count)

# 쿼리 문자열의 개수를 업데이트하여 범주화
df['Query'] = df['url'].apply(Query)

In [ ]:
#9.도메인 생성일 기준 (범주)

# 타임아웃 시간(초) 설정
TIMEOUT = 3

def is_domain_created(url):
    try:
        domain_name = urllib.parse.urlsplit(url).netloc
        socket.setdefaulttimeout(TIMEOUT)  # 타임아웃 설정
        domain_info = whois.whois(domain_name)
        creation_date = domain_info.creation_date

        # creation_date가 list인 경우 첫 번째 요소를 사용
        if isinstance(creation_date, list):
            creation_date = creation_date[0]

        # creation_date가 datetime 형식이 아닌 경우 처리
        if not isinstance(creation_date, datetime):
            return 1  # 피싱 사이트로 간주

        today = datetime.today()
        one_years_ago = today - timedelta(days=365)

        if creation_date <= one_years_ago:
            return 0  # 1년 이상된 경우 피싱 사이트로 간주하지 않음
        else:
            return 1  # 1년 이하(피싱 사이트 의심)
 
    except (whois.parser.PywhoisError, socket.timeout, Exception) as e:
        return 1  # 오류 발생 시 1(피싱 사이트라고 간주)


# 'Domain_Age' 열 추가
df['Domain_Age'] = df['url'].apply(is_domain_created)

In [ ]:
#10.도메인 만료일 기준 (범주)

# 타임아웃 시간(초) 설정
TIMEOUT = 3

# 링크에서 도메인을 추출하는 함수
def get_domain_from_link(link):
    parsed_uri = urlparse(link)
    domain = '{uri.netloc}'.format(uri=parsed_uri)
    return domain

# 도메인 만료 여부를 확인하는 함수
def domain_end(domain_name):
    try:
        socket.setdefaulttimeout(TIMEOUT)
        domain_info = whois.whois(domain_name)
        expiration_date = domain_info.expiration_date

        if isinstance(expiration_date, list):
            expiration_date = expiration_date[0]

        if isinstance(expiration_date, str):
            expiration_date = datetime.strptime(expiration_date, "%Y-%m-%d")

        if expiration_date is None:
            return 0

        if expiration_date.tzinfo is not None:
            expiration_date = expiration_date.replace(tzinfo=None)

        today = datetime.now()
        days_until_expiry = (expiration_date - today).days

        if (days_until_expiry / 30) < 5:
            return 1
        else:
            return 0
    except (whois.parser.PywhoisError, socket.timeout, Exception):
        return 1

# 각 URL에 대해 도메인 만료 여부를 확인하는 함수
def is_domain_created_within_six_months(url):
    domain_name = get_domain_from_link(url)
    return domain_end(domain_name)

# 병렬 처리 함수
def apply_parallel(df, func):
    with concurrent.futures.ThreadPoolExecutor() as executor:
        results = list(executor.map(func, df['url']))
    return results

# 병렬로 'Domain_end' 컬럼 생성 및 함수 적용
df['Domain_end'] = apply_parallel(df, is_domain_created_within_six_months)

In [17]:
#11. Mousover

def check_mouseover(html_content):
    # BeautifulSoup을 사용하여 HTML 파싱
    soup = BeautifulSoup(html_content, 'html.parser')
    # onmouseover 속성을 가진 태그를 찾음
    if soup.find(attrs={"onmouseover": True}):
        return 1  # 마우스 오버 발생
    else:
        return 0  # 마우스 오버 발생 X

def check_url(url):
    try:
        # URL 파싱을 시도하여 유효성 검사
        parsed_url = urlparse(url)
        if not all([parsed_url.scheme, parsed_url.netloc]):
            raise ValueError("Invalid URL")

        response = requests.get(url, timeout=3)  # 타임아웃을 3초로 설정
        response.raise_for_status()  # HTTP 에러가 발생하면 예외 발생
        response_text = response.text
        result = check_mouseover(response_text)
    except (requests.exceptions.RequestException, ValueError) as e:
        result = -1  # 타임아웃 또는 다른 요청 예외 발생 시 -1을 반환
    return result

# 'url' 컬럼의 각 URL에 대해 Mouseover 여부를 확인하여 'Mouseover' 컬럼에 결과를 저장
df['Mouseover'] = df['url'].apply(check_url)

In [18]:
#12. Web_Forwards(범주)
import requests

# 웹 포워딩 체크 함수
def web_forwards(url, timeout=3):
    try:
        respon = requests.get(url, allow_redirects=True, timeout=timeout)
    except requests.RequestException:
        return 1  # 요청 중 오류가 발생하면 피싱 사이트로 간주

    if not respon.history:
        return 0  # 리디렉션이 전혀 없는 경우 의심 사이트

    # 리디렉션 횟수가 2회 이하인 경우 의심 사이트로 간주
    if len(respon.history) <= 2:
        return 0
    else:
        return 1  # 리디렉션이 3회 이상인 경우 피싱 사이트로 간주

# 병렬 처리를 사용하여 각 URL에 대해 web_forwards 함수 실행
def parallel_apply(urls, func, max_workers=5):
    results = []
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        future_to_url = {executor.submit(func, url): url for url in urls}
        for future in as_completed(future_to_url):
            url = future_to_url[future]
            try:
                result = future.result()
            except Exception as exc:
                result = 1  # 예외 발생 시 피싱 사이트로 간주
            results.append(result)
    return results

# 'url' 컬럼의 각 URL에 대해 리디렉션 여부를 확인하여 'Web_forwards' 컬럼에 결과를 저장
df['Web_forwards'] = parallel_apply(df['url'], web_forwards)

In [ ]:
#13. URL이 실제로 갖고 있는 <a> 태그의 수 (범주X)

# 타임아웃 시간(초) 설정
TIMEOUT = 3

def count_hyperlinks(url):
    global a
    try:
        response = requests.get(url, timeout=TIMEOUT)
        response.raise_for_status()  # 요청이 성공했는지 확인
        soup = BeautifulSoup(response.content, 'html.parser')
        hyperlinks = soup.find_all('a')
        
        return len(hyperlinks)
        
    except (requests.RequestException, requests.Timeout):
        return -1  # 요청 실패 시 -1 반환


# 하이퍼링크의 총 개수를 업데이트
df['Hyperlinks'] = df['url'].apply(count_hyperlinks)

In [20]:
#14. 도메인 일관성을 확인하는 함수(범주)

# 도메인 일관성 체크 함수
def check_domain_consistency(url, timeout=3):
    try:
        response = requests.get(url, timeout=timeout)
        original_domain = urlparse(url).netloc
        final_domain = urlparse(response.url).netloc
        is_same_domain = 1 if original_domain == final_domain else 0
        return is_same_domain
    except requests.RequestException:
        return -1

# 병렬 처리를 사용하여 각 URL에 대해 check_domain_consistency 함수 실행
def parallel_apply(urls, func, max_workers=5):
    results = []
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        future_to_url = {executor.submit(func, url): url for url in urls}
        for future in as_completed(future_to_url):
            url = future_to_url[future]
            try:
                result = future.result()
            except Exception as exc:
                result = -1  # 예외 발생 시 -1 반환
            results.append(result)
    return results

# 'url' 컬럼의 각 URL에 대해 도메인 일관성을 확인하여 'Domain_Cons' 컬럼에 결과를 저장
df['Domain_Cons'] = parallel_apply(df['url'], check_domain_consistency)

In [21]:
from urllib.parse import urlparse, parse_qs

def tokenize_url(url: str):
    """
    주어진 URL을 토큰화하여 리스트로 반환하는 함수.

    Parameters:
    url (str): 토큰화할 URL 문자열.

    Returns:
    list: URL의 토큰 리스트.
    """
    tokens = []
    # URL 파싱
    parsed_url = urlparse(url)
    
    # 도메인 토큰화
    domain_tokens = parsed_url.netloc.split('.')
    tokens.extend(domain_tokens)
    
    # 경로 토큰화
    path_tokens = parsed_url.path.split('/')
    tokens.extend([token for token in path_tokens if token])
    
    # 쿼리 매개변수 토큰화
    query_tokens = parse_qs(parsed_url.query)
    for key, values in query_tokens.items():
        tokens.append(key)
        tokens.extend(values)
    
    return tokens

df['Tokenized_url'] = df['url'].apply(lambda x: ' '.join(tokenize_url(x)))

In [22]:
df['Label'] = 1

In [23]:
df.to_csv('valid_phish_data.csv', index=False)

In [24]:
df

,url,IP_LIKE,AT,URL_Depth,Redirection,Is_Https,TINY_URL,Check_Hyphen,Query,Domain_Age,Domain_end,Mouseover,Web_forwards,Hyperlinks,Domain_Cons,Tokenized_url,Label
0,https://allegrolokalnie.oferta-65492.pl/oferta...,0.0,0.0,1.0,0.0,1,0.0,1,0,1,0,-1,1,-1,-1,allegrolokalnie oferta-65492 pl oferta l Jeden...,1
1,https://pub-d0d0d5916dbd4da5b9fe7ed070e51dab.r...,0.0,0.0,0.0,0.0,1,0.0,1,-1,0,0,0,1,-1,-1,pub-d0d0d5916dbd4da5b9fe7ed070e51dab r2 dev ar...,1
2,https://anmeldung.client-eportal.is-certified....,0.0,0.0,1.0,0.0,1,0.0,1,-1,1,1,-1,1,-1,-1,anmeldung client-eportal is-certified com acce...,1
3,http://malerbetrieb-steinin.blogdns.org/.kunde...,0.0,0.0,0.0,0.0,0,0.0,1,-1,1,0,-1,1,-1,-1,malerbetrieb-steinin blogdns org .kundenarea,1
4,https://aktualisieren.83-229-87-162.cprapid.co...,0.0,0.0,1.0,0.0,1,0.0,1,-1,0,0,0,1,-1,-1,aktualisieren 83-229-87-162 cprapid com aktf43...,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,http://www.gwenet.org/,0.0,0.0,-1.0,0.0,0,0.0,0,-1,0,1,-1,1,-1,1,www gwenet org,1
19996,http://files.planetaryedges.workers.dev,0.0,0.0,-1.0,0.0,0,0.0,0,-1,1,0,-1,0,-1,1,files planetaryedges workers dev,1
19997,https://docs.google.com/presentation/d/e/2PACX...,0.0,0.0,1.0,0.0,1,0.0,1,1,0,0,0,0,0,1,docs google com presentation d e 2PACX-1vRgVRg...,1
19998,https://docs.google.com/presentation/d/e/2PACX...,0.0,0.0,1.0,0.0,1,0.0,1,1,0,0,0,0,0,1,docs google com presentation d e 2PACX-1vTEHkE...,1
